In [2]:
# Parameters (DO NOT containerize this cell)

# Parameters
param_s3_server = "scruffy.lab.uvalight.net:9000"
param_copernicus_e_mail_general_user_account = ""  # E-mail of a general user account for Copernicus. Create one here: https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings
param_s3_public_bucket = "naa-vre-waddenzee-shared"

# Secrets
secret_copernicus_password_general_user_account = ""  # password of a general user account for Copernicus.
secret_s3_secret_key = ""
secret_s3_access_key = ""

In [3]:
!pwd

/home/jovyan


In [4]:
# RS-retrieving-Py01

import json
import os, sys
from dtSat import dtSat
from dtAcolite import dtAcolite
import glob
from minio import Minio
import acolite as ac

year = 2016 
start_date = f"{year}-01-01"
end_date   = f"{year}-12-31"
data_collection = "SENTINEL-2"
product_type = "S2MSI1C"
aoi = "POLYGON((4.6 53.1, 4.9 53.1, 4.9 52.8, 4.6 52.8, 4.6 53.1))'"
collection = "sentinel"

app_configuration = dtAcolite.configure_acolite_directory(base_dir = "/tmp/data", year = year, collection = collection)
# app_configuration = dtAcolite.load_configuration("./")
print(app_configuration)

catalogue_response = dtSat.get_sentinel_catalogue(start_date, end_date, data_collection = data_collection, aoi= aoi, product_type=product_type, cloudcover=10.0, max_results=1000)
catalogue_sub = dtSat.filter_by_orbit_and_tile(catalogue_response, orbit = "R051", tile = "T31UFU", name_only = False)


catalogue_sub_filename = "/tmp/data/catalogue_sub.json"
with open(catalogue_sub_filename, 'w') as f:
    json.dump(catalogue_sub, f)
    
access_response = dtSat.get_copernicus_access_token(username = param_copernicus_e_mail_general_user_account, 
                                                    password = secret_copernicus_password_general_user_account)

access_response_filename = "/tmp/data/access_response.json"
with open(access_response_filename, 'w') as f:
    json.dump(access_response, f)
    
print(f"Raw images will be stored in {app_configuration['raw_inputdir']}")
print(f"Analysis with acolite will start with images from {app_configuration['acolite_inputdir']}")
print(f"Processed images from acolite will be stored in {app_configuration['acolite_outputdir']}")

## cell-2

with open(catalogue_sub_filename) as f:
    catalogue_sub = json.load(f)
    
with open(access_response_filename) as f:
    access_response = json.load(f)
    
dtSat.data_sentinel_request(access_response, 
                            catalogue_sub, 
                            dir_path = app_configuration["raw_inputdir"])


print(f"List of all images downloaded in {app_configuration['raw_inputdir']} included: ")
print(os.listdir(app_configuration["raw_inputdir"]))

# cell-3

minio_client = Minio(param_s3_server, access_key=secret_s3_access_key, secret_key=secret_s3_secret_key, region = "nl-uvalight", secure=True)
minio_client

minio_base_path = "app_acolite_qing"
dtSat.upload_satellite_to_minio(client = minio_client,
                                bucket_name = param_s3_public_bucket,  
                                local_path = app_configuration["raw_inputdir"],
                                minio_path = f"/{minio_base_path}/raw/{app_configuration['collection']}/{app_configuration['year']}", 
                                collection = app_configuration["raw_inputdir"], 
                                year = app_configuration["raw_inputdir"])


inputfilenames = dtAcolite.create_acolite_input(app_configuration = app_configuration)
outfilepaths   = dtAcolite.create_acolite_output(app_configuration=app_configuration, filenames=inputfilenames)
dtAcolite.unzip_inputfiles(app_configuration=app_configuration)

inputfilepaths = glob.glob(f"{app_configuration['acolite_inputdir']}/**")
outputfilepaths = glob.glob(f"{app_configuration['acolite_outputdir']}/**")
outputfilepaths
inputfilepaths

path_ids = list(range(len(inputfilepaths)))

ModuleNotFoundError: No module named 'dtSat'

In [ ]:
# RS-processing-Py01

import acolite as ac

year = 2016
start_date = f"{year}-01-01"
end_date   = f"{year}-12-31"
data_collection = "SENTINEL-2"
product_type = "S2MSI1C"
aoi = "POLYGON((4.6 53.1, 4.9 53.1, 4.9 52.8, 4.6 52.8, 4.6 53.1))'"
collection = "sentinel"

app_configuration = dtAcolite.configure_acolite_directory(base_dir = "/tmp/data", year = year, collection = collection)
inputfilepaths = glob.glob(f"{app_configuration['acolite_inputdir']}/**")
outputfilepaths = glob.glob(f"{app_configuration['acolite_outputdir']}/**")
# settings
settings = {'limit': [52.5,4.7,53.50,5.4], 
            'inputfile': '', 
            'output': '', 
            "cirrus_correction": True,
            'l2w_parameters' : ["rhow_*","rhos_*", "Rrs_*", "chl_oc3", "chl_re_gons", "chl_re_gons740", 
                                "chl_re_moses3b", "chl_re_moses3b740",  "chl_re_mishra", "chl_re_bramich", 
                                "ndci", "ndvi","spm_nechad2010"]}

acolite_processing = []
for i in path_ids:
    print("---------------------------------------------------------------------------------------")
    settings['inputfile'] = inputfilepaths[i]
    settings['output']    = outputfilepaths[i]
    ac.acolite.acolite_run(settings=settings)

    # this should be modified to verify acolite processing, and for merging feature in workflow 
    message = f"processing done and output is in {inputfilepaths[i]}"
    acolite_processing.append(message)

    # print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print(f"processing done and output is in {inputfilepaths[i]}")
# print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print(acolite_processing)


In [ ]:
# RS-cell-3-upload
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - acolite_processing: List
#   outputs: []
#   params:
#    - param_s3_server: String
#    - param_copernicus_e_mail_general_user_account: String  # E-mail of a general user account for Copernicus. Create one here: https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings
#    - param_s3_public_bucket: String
#    - year: Integer
#   secrets:
#    - secret_copernicus_password_general_user_account: String  # password of a general user account for Copernicus.
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
# ...

acolite_processing

In [ ]:
### do parallezation



In [ ]:
from reprolab.environment import freeze_venv_dependencies
freeze_venv_dependencies('my_venv')